https://developer.spotify.com/documentation/web-api/

https://developer.spotify.com/

https://developer.spotify.com/dashboard/login

**Objectives:**
- recommend the next song for an user base on his current playlist

**Assumptions:**
- tracks of a given playlist are the favorite songs of the user

**Methodology:**
1. Collaborative filtering
2. Initial sample for each user will take 70% of the tracks in the playlist (check if the sorting of the musics in the playlist has any information - date of inclusion, user sorting, etc) and then use the remaining 30% of the tracks as test.
3. **If we have time:** Hybrid model by adding *Content based filtering* for playlists with few tracks
    - Track features: genre, time, top_track, etc.
**Concerns:**
- cold start: playlists with few tracks
- rare tracks: treatment to be applied to the tracks with low frequency on the database
- only positive rating: we will have only tracks that the user liked, we do not have any data about tracks that they did not like.


**Python Libraries:**
- Sagemaker
- Elasticsearch
- PySpark

In [ ]:
# Default libraries: pandas, numpy
# import beakerx
import cProfile
import collections
import datetime

import feather
import itertools
import math
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# import urbangrammar_graphics as ugg

from IPython.display import Markdown as md
from collections import Counter
from clustergram import Clustergram
from datetime import datetime
from dotenv import load_dotenv
from itertools import groupby

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, BaggingClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_validate, learning_curve, GridSearchCV, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})

In [ ]:
def single_list(my_list):
    single = []
    for e in my_list:
        single.extend(e)
    return single

def single_list_nested(my_list):
    single = []
    for e in my_list:
        for a in e:
            single.extend(a)
    return single

def sorted_list(xs):
    counts = collections.defaultdict(int)
    for x in xs:
        counts[x] += 1
    return sorted(counts.items(), reverse=True, key=lambda tup: tup[1])

In [ ]:
data_path = os.getcwd() + '\\data\\external\\raw_data.pkl'
df = pd.read_pickle(data_path)
print(df.info())

# total = 0
# idx = 0
# while total < 10000:
# #     print(len(df.iloc[idx, 2]))
    
#     total = total + len(df.iloc[idx, 2])
#     idx = idx + 1
# #     len(df.iloc[0:n, :])
# print(idx)

In [ ]:
def playlist_to_row(playlist, track_matrix):
    for col_track in playlist:
                if col_track is not None:
                    col = s_tracks_id.index(col_track)
                    track_matrix[row, col] += 1

In [ ]:
def ranking(df,id_column):
    single_list_temp = single_list(df[id_column])
    data = {id_column: single_list_temp}
    aux_df = pd.DataFrame(data)
    
    name = id_column.replace('_id','_count')
    ranked_sorted = aux_df.groupby([id_column]).agg(
        temp_count=pd.NamedAgg(column=id_column, aggfunc="count")
    )
    ranked_sorted = ranked_sorted.rename(columns={'temp_count': name})
    ranked_sorted = ranked_sorted.sort_values(name,ascending=False)
    return ranked_sorted
# ranking(df_test,'tracks_id')
# print(tracks_rank)

def top(ranked,n):
    top_n = ranked.iloc[:n]
    
    return top_n
# tracks_rank = ranking(df_test,'tracks_id')
# top(tracks_rank,10)


In [ ]:
# matrix track x track
# idx = 131
# df_test = df.iloc[0:idx, :]
df_test = df

print(df_test.info())

s_tracks_id = single_list(df_test['tracks_id'])
data = {'tracks_id': s_tracks_id}
df_track = pd.DataFrame(data)

tracks_rank = df_track.groupby(['tracks_id']).agg(
    tracks_count=('tracks_id','count')
    )
tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

# tracks_rank


# s_tracks_id = list(set(single_list(df_test['tracks_id'])))
# s_tracks_id = list(set(single_list(df['tracks_id'])))
s_tracks_id = list(tracks_rank.index)
# print(len(s_tracks_id))
# s_tracks = single_list(df_test['tracks'])

shape = [len(s_tracks_id),len(s_tracks_id)]
track_matrix = np.zeros(shape, dtype=int, order='C')

for i, row_track in enumerate(s_tracks_id):
    row = s_tracks_id.index(row_track)
    if i != row: print("i: " + str(i) + ": " + row_track), print()
    
    
    for playlist in df_test['tracks_id']:
        try:
            check = playlist.index(row_track)
        except ValueError:
            check = -1
        if check >= 0:
            for col_track in playlist:
                if col_track is not None:
                    col = s_tracks_id.index(col_track)
#                     print(col_track  + " - " + str(col))
#                     tr = df_test['tracks'][n][col]
#                     print("track: " + str(n) + ": " + col_track + " - col_match:" + str(col) + ": ") # + tr)
                    track_matrix[row, col] += 1
# print(track_matrix)

In [ ]:
track_matrix

In [ ]:
# compare principal diagonal versus the counting of each track
# print(tracks_rank)
n_rows, n_cols = track_matrix.shape
for i in range(0,n_rows):
    for j in range(i,n_cols):
        if track_matrix[i][j] != track_matrix[j][i]:
            print("Difference: " + str(i) + "," + str(j))
# for i, in enumerate(track_matrix):
#     for j,col in enumerate(row):
#         if 

In [ ]:
# Check if the matrix is symmetrical
column_sums = track_matrix.sum(axis=0)
row_sums = track_matrix.sum(axis=1)

print(tracks_rank)
print(column_sums)
print(row_sums)

In [ ]:
if len(row_sums) != len(column_sums):
    print("error row * col")

for i in range(0,len(row_sums))
     if tracks_rank[i] != column_sums[i]:
            print("Difference: " + str(i) + "," + str(j))
print(tracks_rank)
print(column_sums)

In [ ]:
# Check if the sum of rows matches the sum of the columns
column_sums = track_matrix.sum(axis=0)
row_sums = track_matrix.sum(axis=1)

print(column_sums)
print(row_sums)

if len(column_sums) != len(row_sums):
    print("error col * row")

for i in range(0,len(row_sums)):
     if row_sums[i] != column_sums[i]:
            print("Difference: " + str(i) + "," + str(j))

In [ ]:
# min_max = MinMaxScaler()
# tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

track_normed_by_col = track_matrix / track_matrix.max(axis=0)
track_normed_by_row = track_matrix / track_matrix.max(axis=1)

delta = track_normed_by_col - track_normed_by_row
print(delta)

In [ ]:
fig = plt.figure(figsize=(20, 20))

ax = fig.add_subplot(111)
plt.imshow(-track_matrix, interpolation='nearest', cmap=plt.cm.gist_heat)
ax.set_aspect('equal')
plt.show()

In [ ]:
load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
def get_playlist_detail(user_id, playlist_id):
    l_artists = []
    l_albums = []
    l_tracks = []
    l_popularity = []
    l_artists_id = []
    l_albums_id = []
    l_tracks_id = []
    
    playlist = sp.user_playlist(user_id, playlist_id)
    
    for item in playlist['tracks']['items']:
        group = []
        group_id = []
        for artist in item['track']['album']['artists']:
            group.append(artist['name'])
            group_id.append(artist['id'])
        l_artists.append(group)
        l_artists_id.append(group_id)
        l_albums.append(item['track']['album']['name'])
        l_tracks.append(item['track']['name'])
        l_popularity.append(item['track']['popularity'])
        
        l_albums_id.append(item['track']['album']['id'])
        l_tracks_id.append(item['track']['id'])
        
    return user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id;

# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id = get_playlist_detail('spotify', '37i9dQZF1DWXjlWxUbm84A')
# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_1_uid_pid.pkl'
if os.path.exists(data_path):
    data_path_2 = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            uid_pid_raw = pickle.load(f)
else:
    n = 950 # max = 950
    p_limit = 50 # limit of playlists max = 50

    # list of all user ids (UID) and playlist ids (PID)
    uid_pid_raw = []
    
    for y in range(2018,2022,1):
        txt = 'year:' + str(y)
        for i in range(0,n,1):
            if i % 100 == 0: print(txt + ': ' + str(i) + '/' + str(n))
            uid_pid = sp.search(q=txt, type='playlist', limit=p_limit, offset=i) #,market='US'
            uid_pid_raw.append(uid_pid)
    # Save the data
    with open(data_path, 'wb') as f:
        pickle.dump(uid_pid_raw, f)


t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
with open(data_path, 'wb') as f:
        pickle.dump(uid_pid_raw, f)

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
if os.path.exists(data_path):
#     print("Loading data...")
#     data_path_2 = os.getcwd() + '\\data\\external\\split_1_playlists.pkl'
#     if not(os.path.exists(data_path_2)):
#         with open(data_path, 'rb') as f:
#             playlist_raw = pickle.load(f)
else:
    print("Requesting playlists - Spotipy")
#     playlist_raw = []

#     for i, result in enumerate(uid_pid_raw):
#         print(i)
#         if i % 100 == 0: print(i)
#         for user in result['playlists']['items']:
#             pid = user['id']
#             uid = user['owner']['id']
#             try:
#                 playlist = sp.user_playlist(user['id'],user['owner']['id'])
#                 playlist_raw.append(playlist)
#             except:
#                 print(uid + " " + pid)
                   
   # Save the data
#     with open(data_path, 'wb') as f:
#         pickle.dump(playlist_raw, f)

t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_data.pkl'
if os.path.exists(data_path):
    df = pd.read_pickle(data_path)
else:
#     print('ops')
    n = 50
    p_limit = 50 # limit of playlists max = 50

    user_id = []
    playlist_id = []
    artists = []
    albums = []
    tracks = []
    popularity = []
    artists_id = []
    albums_id = []
    tracks_id = []

    for i in uid_pid:
        if i-1 % 100 == 0: print(i)
        for user in result['playlists']['items']:
            pid = user['id']
            uid = user['owner']['id']
#                 user_id.append(uid)
#                 playlist_id.append(pid)
            p_user_id, p_playlist_id, p_artists, p_albums, p_tracks, p_popularity, p_artists_id, p_albums_id, p_tracks_id = get_playlist_detail(uid,pid)

            user_id.append(p_user_id)
            playlist_id.append(p_playlist_id)
            artists.append(p_artists)
            albums.append(p_albums)
            tracks.append(p_tracks)
            popularity.append(p_popularity)
            artists_id.append(p_artists_id)
            albums_id.append(p_albums_id)
            tracks_id.append(p_tracks_id)
#             if i-1 % 100 == 0: print('tot:' + str(len(playlist_id)))

    # intialise data of lists.
    data = {
        'user_id': user_id,
        'playlist_id': playlist_id,
        'tracks_id': tracks_id,
        'tracks': tracks,
        'popularity': popularity,
        'artists_id': artists_id,
        'artists': artists,
        'albums_id': albums_id,
        'albums': albums
    }

    # Create DataFrame
    df = pd.DataFrame(data)
    df.head()

    # Save the data
    df.to_pickle(data_path)

t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
s_tracks_id = single_list(df['tracks_id'])
s_tracks = single_list(df['tracks'])
s_popularity = single_list(df['popularity'])

s_artists = single_list_nested(df['artists'])
s_artists_id = single_list_nested(df['artists_id'])

s_albums = single_list(df['albums'])
s_albums_id = single_list(df['albums_id'])

data = {
    'tracks_id': s_tracks_id,
    'tracks': s_tracks,
    'popularity': s_popularity
}
df_track = pd.DataFrame(data)
# print(df_track.head())

data = {
    'artists_id': s_artists_id,
    'artists': s_artists
}
df_artist = pd.DataFrame(data)
# print(df_artist.head())


data = {
    'albums_id': s_albums_id,
    'albums': s_albums
}
df_album = pd.DataFrame(data)
# print(df_album.head())

In [ ]:
tracks_rank = df_track.groupby(['tracks_id','tracks']).agg(
    popularity=('popularity','max'),
    tracks_count=('tracks_id','count')
    )

min_max = MinMaxScaler()
tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

tracks_rank["rank"] = tracks_rank['tracks_count'].rank(method='average',ascending=False)

tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

tracks_rank

In [ ]:
album_rank = df_album.groupby(['albums_id','albums']).agg(album_count=('albums_id','count'))

min_max = MinMaxScaler()
album_rank['count_scaled'] = min_max.fit_transform(album_rank[["album_count"]])

album_rank["rank"] = album_rank['album_count'].rank(method='average',ascending=False)

album_rank = album_rank.sort_values('album_count',ascending=False)

album_rank

In [ ]:
df_artist.head
artist_rank = df_artist.groupby(['artists_id','artists']).agg(artist_count=('artists_id','count'))

min_max = MinMaxScaler()
artist_rank['count_scaled'] = min_max.fit_transform(artist_rank[["artist_count"]])

artist_rank["rank"] = artist_rank['artist_count'].rank(method='average',ascending=False)

artist_rank = artist_rank.sort_values('artist_count',ascending=False)

artist_rank

In [ ]:
# tracks_cartesian = []
# for element in itertools.product(s_tracks_id,s_tracks_id):
#     tracks_cartesian.append(element)

# https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
# def cartesian_product(*arrays):
#     la = len(arrays)
#     dtype = np.result_type(*arrays)
#     arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
#     for i, a in enumerate(numpy.ix_(*arrays)):
#         arr[...,i] = a
#     return arr.reshape(-1, la)


# # tracks_cartesian = cartesian_product(*(s_tracks_id * 2))

In [ ]:
# for key, item in df_test.items():
#     print(key)
#     if key == 'tracks_id':
#         print(item)

In [ ]:
# # Define count_entries()
# def count_entries(csv_file,c_size,colname):
#     """Return a dictionary with counts of
#     occurrences as value for each key."""
    
#     # Initialize an empty dictionary: counts_dict
#     counts_dict = {}

#     # Iterate over the file chunk by chunk
#     for chunk in pd.read_csv(csv_file,chunksize=c_size):

#         # Iterate over the column in DataFrame
#         for entry in chunk[colname]:
#             if entry in counts_dict.keys():
#                 counts_dict[entry] += 1
#             else:
#                 counts_dict[entry] = 1

#     # Return counts_dict
#     return counts_dict

# # Call count_entries(): result_counts
# result_counts = count_entries('tweets.csv',10,'lang')

# # Print result_counts
# print(result_counts)


In [ ]:
from numba import jit, cuda
# from numba import cuda
import numpy as np
import code
# to measure exec time
from timeit import default_timer as timer
  
# normal function to run on cpu
def func(a):                                
    for i in range(10000000):
        a[i]+= 1

# function optimized to run on gpu 
cuda.jit(target ="cuda")                         
code.interact(local=locals)
def func2(a):
    for i in range(100000000):
        a[i]+= 1

if __name__=="__main__":
    n = 10000000                            
    a = np.ones(n, dtype = np.float64)
    b = np.ones(n, dtype = np.float32)
      
    start = timer()
    func(a)
    print("without GPU:", timer()-start)    
      
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

In [ ]:
from numba import jit, cuda
# from numba import cuda
import numpy as np
from timeit import default_timer as timer

@cuda.jit(device=True)
def mandel(x, y, max_iters):
  """
  Given the real and imaginary parts of a complex number,
  determine if it is a candidate for membership in the Mandelbrot
  set given a fixed number of iterations.
  """
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 4:
      return i

  return max_iters

@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
  startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    real = min_x + x * pixel_size_x
    for y in range(startY, height, gridY):
      imag = min_y + y * pixel_size_y 
      image[y, x] = mandel(real, imag, iters)

gimage = np.zeros((1024, 1536), dtype = np.uint8)
blockdim = (32, 8)
griddim = (32,16)

start = timer()
d_image = cuda.to_device(gimage)
mandel_kernel[griddim, blockdim](-2.0, 1.0, -1.0, 1.0, d_image, 20) 
d_image.to_host()
dt = timer() - start

print("Mandelbrot created on GPU in %f s" % dt)

imshow(gimage)

In [ ]:
# Scale by row
# min_max = MinMaxScaler()
# tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

track_normed = track_matrix / track_matrix.max(axis=0)
track_normed_by_row = track_matrix / track_matrix.max(axis=1)

delta = track_normed - track_normed_by_row
print(delta.sum())

In [ ]:
# graphical visualization
fig = plt.figure(figsize=(20, 20))

ax = fig.add_subplot(111)
# plt.imshow(track_matrix, interpolation='nearest', cmap=plt.cm.gist_heat)
plt.imshow(track_normed, interpolation='nearest', cmap=plt.cm.Purples)
ax.set_aspect('equal')
plt.show()

In [ ]:
def get_playlist_detail(user_id, playlist_id):
    l_artists = []
    l_albums = []
    l_tracks = []
    l_popularity = []
    l_artists_id = []
    l_albums_id = []
    l_tracks_id = []
    
    playlist = sp.user_playlist(user_id, playlist_id)
    
    for item in playlist['tracks']['items']:
#         group = []
#         group_id = []
        for artist in item['track']['album']['artists']:
            group.append(artist['name'])
            group_id.append(artist['id'])
        l_artists.append(group)
        l_artists_id.append(group_id)
        l_albums.append(item['track']['album']['name'])
        l_tracks.append(item['track']['name'])
        l_popularity.append(item['track']['popularity'])
        
        l_albums_id.append(item['track']['album']['id'])
        l_tracks_id.append(item['track']['id'])
        
    playlist_details = zip(user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id)
    return playlist_details;

# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id = get_playlist_detail('spotify', '37i9dQZF1DWXjlWxUbm84A')
# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id

In [ ]:
# temp = df_test[['tracks_id','tracks']]
# print(temp)

# for i, a in enumerate(temp['tracks_id']):
#     print(i)
#     print(a)

# for i in df_test:
#     print(i)

# print(temp['tracks'][1])
# df_test[['tracks','tracks_id','popularity']].iloc[1]

In [ ]:
def dict_id_name(data, id_column, name_column):
    dict_id_name = dict()

    for i, item in enumerate(data[[id_column, name_column]][id_column]):
        dict_id_name = dict(
            zip(item, data[[id_column, name_column]][name_column][i]))
    return dict_id_name


# dict_id_name(df_test,'tracks_id','tracks')


def dict_tracks(data):
    dict_id_name = dict()

    for i, item in enumerate(data['tracks_id']):
#     for item in data['tracks_id']:
        print(i)
#         print(item)
#         print(data.iloc[i])
        
        for k in item:
            print(k)
        
        
#         print(
#             data[[
#                 'tracks', 'popularity', 'artists_id', 'artists', 'albums_id',
#                 'albums'
#             ]].iloc[i])
#         dict_id_name = dict(zip(item, list(data[['tracks']].iloc[i])))
#         dict_id_name1 = dict(zip(item, data[['popularity']].iloc[i]))
#         dict_id_name2 = dict(zip(item, data[['artists_id']].iloc[i]))
#         dict_id_name3 = dict(zip(item, data[['artists']].iloc[i]))
#         dict_id_name4 = dict(zip(item, data[['albums_id']].iloc[i]))
#         dict_id_name5 = dict(zip(item, data[['albums']].iloc[i]))

#         for grp in groups:
#             #do some calcs to get a dataframe called 'df'
#             frames[grp] = df
#         for k in item['tracks_id']:
#             print(k)
#             print(data[['tracks', 'popularity', 'artists_id', 'artists', 'albums_id',
#                 'albums']].iloc[i][k])
#             dict_id_name[k] = data[[
#                 'tracks', 'popularity', 'artists_id', 'artists', 'albums_id',
#                 'albums'
#             ]].iloc[k]
#             dict_id_name = dict(zip(item, list(data[['tracks']].iloc[i])))
    return dict_id_name
        

print(dict_tracks(df_test[:2]))

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_1_uid_pid.pkl'
if os.path.exists(data_path):
    data_path_2 = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            uid_pid_raw = pickle.load(f)
else:
    n = 950 # max = 950
    p_limit = 50 # limit of playlists max = 50

    # list of all user ids (UID) and playlist ids (PID)
    uid_pid_raw = []
    
    for y in range(2018,2022,1):
        txt = 'year:' + str(y)
        for i in range(0,n,1):
            if i % 100 == 0: print(txt + ': ' + str(i) + '/' + str(n))
            uid_pid = sp.search(q=txt, type='playlist', limit=p_limit, offset=i,market='US')
            uid_pid_raw.append(uid_pid)
    # Save the data
    with open(data_path, 'wb') as f:
        pickle.dump(playlist_raw, f)


t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
if os.path.exists(data_path):
    print("Loading data...")
    data_path_2 = os.getcwd() + '\\data\\external\\split_1_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            playlist_raw = pickle.load(f)
else:
    print("Requesting playlists - Spotipy")
#     playlist_raw = []

#     for i, result in enumerate(uid_pid_raw):
#         print(i)
#         if i % 100 == 0: print(i)
#         for user in result['playlists']['items']:
#             pid = user['id']
#             uid = user['owner']['id']
#             try:
#                 playlist = sp.user_playlist(user['id'],user['owner']['id'])
#                 playlist_raw.append(playlist)
#             except:
#                 print(uid + " " + pid)
                   
   # Save the data
#     with open(data_path, 'wb') as f:
#         pickle.dump(playlist_raw, f)

t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
t_start = time.time()
data_path = os.getcwd() + '\\data\\external\\raw_data.pkl'
if os.path.exists(data_path):
    df = pd.read_pickle(data_path)
else:
#     print('ops')
    n = 50
    p_limit = 50 # limit of playlists max = 50

    user_id = []
    playlist_id = []
    artist = []
    album = []
    track = []
    popularity = []
    artist_id = []
    album_id = []
    track_id = []

    for i in uid_pid:
        if i-1 % 100 == 0: print(i)
        for user in result['playlists']['items']:
            pid = user['id']
            uid = user['owner']['id']
#                 user_id.append(uid)
#                 playlist_id.append(pid)
            p_user_id, p_playlist_id, p_artist, p_album, p_track, p_popularity, p_artists_id, p_albums_id, p_tracks_id = unzip(get_playlist_detail(uid,pid))

            user_id.append(p_user_id)
            playlist_id.append(p_playlist_id)
            artist.append(p_artist)
            album.append(p_album)
            track.append(p_track)
            popularity.append(p_popularity)
            artist_id.append(p_artist_id)
            album_id.append(p_album_id)
            track_id.append(p_track_id)
#             if i-1 % 100 == 0: print('tot:' + str(len(playlist_id)))

    # intialise data of lists.
    data = {
        'user_id': user_id,
        'playlist_id': playlist_id,
        'track_id': track_id,
        'track': track,
        'popularity': popularity,
        'artist_id': artist_id,
        'artist': artist,
        'album_id': albums_id,
        'album': album
    }

    # Create DataFrame
    df = pd.DataFrame(data)
    df.head()

    # Save the data
    df.to_pickle(data_path)

t_end = time.time()
np.abs(t_start - t_end)

In [ ]:
s_tracks_id = single_list(df['tracks_id'])
s_tracks = single_list(df['tracks'])
s_popularity = single_list(df['popularity'])

s_artists = single_list_nested(df['artists'])
s_artists_id = single_list_nested(df['artists_id'])

s_albums = single_list(df['albums'])
s_albums_id = single_list(df['albums_id'])

data = {
    'tracks_id': s_tracks_id,
    'tracks': s_tracks,
    'popularity': s_popularity
}
df_track = pd.DataFrame(data)
# print(df_track.head())

data = {
    'artists_id': s_artists_id,
    'artists': s_artists
}
df_artist = pd.DataFrame(data)
# print(df_artist.head())


data = {
    'albums_id': s_albums_id,
    'albums': s_albums
}
df_album = pd.DataFrame(data)
# print(df_album.head())

In [ ]:
tracks_rank = df_track.groupby(['tracks_id','tracks']).agg(
    popularity=('popularity','max'),
    tracks_count=('tracks_id','count')
    )

min_max = MinMaxScaler()
tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

tracks_rank["rank"] = tracks_rank['tracks_count'].rank(method='average',ascending=False)

tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

tracks_rank

In [ ]:
album_rank = df_album.groupby(['albums_id','albums']).agg(album_count=('albums_id','count'))

min_max = MinMaxScaler()
album_rank['count_scaled'] = min_max.fit_transform(album_rank[["album_count"]])

album_rank["rank"] = album_rank['album_count'].rank(method='average',ascending=False)

album_rank = album_rank.sort_values('album_count',ascending=False)

album_rank

In [ ]:
df_artist.head
artist_rank = df_artist.groupby(['artists_id','artists']).agg(artist_count=('artists_id','count'))

min_max = MinMaxScaler()
artist_rank['count_scaled'] = min_max.fit_transform(artist_rank[["artist_count"]])

artist_rank["rank"] = artist_rank['artist_count'].rank(method='average',ascending=False)

artist_rank = artist_rank.sort_values('artist_count',ascending=False)

artist_rank

In [ ]:
# for key, item in df_test.items():
#     print(key)
#     if key == 'tracks_id':
#         print(item)

In [ ]:
# # Define count_entries()
# def count_entries(csv_file,c_size,colname):
#     """Return a dictionary with counts of
#     occurrences as value for each key."""
    
#     # Initialize an empty dictionary: counts_dict
#     counts_dict = {}

#     # Iterate over the file chunk by chunk
#     for chunk in pd.read_csv(csv_file,chunksize=c_size):

#         # Iterate over the column in DataFrame
#         for entry in chunk[colname]:
#             if entry in counts_dict.keys():
#                 counts_dict[entry] += 1
#             else:
#                 counts_dict[entry] = 1

#     # Return counts_dict
#     return counts_dict

# # Call count_entries(): result_counts
# result_counts = count_entries('tweets.csv',10,'lang')

# # Print result_counts
# print(result_counts)
